In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
#import congitive assessment spreadsheet
#path = "./Data/matthewl225_7_15_2019_16_11_22.csv"
path = input('Path to cognitive assesment csv: ')
header = ['ADRC_ADRCCLINICALDATA ID', 'dx1']

#dfcolumns = pd.read_csv(path, nrows = 1)
df = pd.read_csv(path, header = None, skiprows = 1, usecols = [0,8], names = header)

#df = pd.read_csv(path, names=header, index_col=False)

Path to cognitive assesment csv: ./Data/matthewl225_7_15_2019_16_11_22.csv


In [3]:
df.shape

(6224, 2)

In [4]:
df[:3]

,ADRC_ADRCCLINICALDATA ID,dx1
0,OAS30001_ClinicalData_d0000,Cognitively normal
1,OAS30001_ClinicalData_d0339,Cognitively normal
2,OAS30001_ClinicalData_d0722,Cognitively normal


In [5]:
df.dx1.unique()

array(['Cognitively normal', 'uncertain dementia',
       'Unc: ques. Impairment', 'AD dem w/depresss- not contribut',
       'AD Dementia', 'AD dem distrubed social- with',
       'AD dem w/CVD contribut', 'Non AD dem- Other primary',
       'AD dem visuospatial- with', 'AD dem visuospatial- prior',
       'AD dem Language dysf after', '.',
       'uncertain- possible NON AD dem', 'Vascular Demt- primary',
       'AD dem w/PDI after AD dem not contrib', '0.5 in memory only',
       'Incipient Non-AD dem', 'AD dem distrubed social- prior',
       'Frontotemporal demt. prim', 'AD dem distrubed social- after',
       'AD dem w/PDI after AD dem contribut',
       'uncertain  possible NON AD dem',
       'AD dem w/depresss  not contribut',
       'AD dem w/oth (list B) contribut', 'DLBD- primary',
       'Incipient demt PTP', 'Dementia/PD- primary',
       'AD dem w/depresss- contribut', 'Unc: impair reversible',
       'AD dem w/oth (list B) not contrib', 'AD dem w/CVD not contrib',
     

In [35]:
df.dx1.isna().sum()

2

In [7]:
import re

subject_regex = re.compile("OAS(?P<order>[0-9]+)")
subjects = [subject_regex.search(r).group(1) for r in df["ADRC_ADRCCLINICALDATA ID"]]
#print(subjects)
df['Subject'] = pd.Series(subjects, index=df.index)

date_regex = re.compile("d(?P<order>[0-9]+)")
dates = [date_regex.search(r).group(1) for r in df["ADRC_ADRCCLINICALDATA ID"]]
df['Date'] = pd.Series(list(map(int,dates)), index=df.index)

In [8]:
df[:15]

,ADRC_ADRCCLINICALDATA ID,dx1,Subject,Date
0,OAS30001_ClinicalData_d0000,Cognitively normal,30001,0
1,OAS30001_ClinicalData_d0339,Cognitively normal,30001,339
2,OAS30001_ClinicalData_d0722,Cognitively normal,30001,722
3,OAS30001_ClinicalData_d1106,Cognitively normal,30001,1106
4,OAS30001_ClinicalData_d1456,Cognitively normal,30001,1456
5,OAS30001_ClinicalData_d1894,Cognitively normal,30001,1894
6,OAS30001_ClinicalData_d2181,Cognitively normal,30001,2181
7,OAS30001_ClinicalData_d2699,Cognitively normal,30001,2699
8,OAS30001_ClinicalData_d3025,Cognitively normal,30001,3025
9,OAS30001_ClinicalData_d3332,Cognitively normal,30001,3332


In [9]:
import scipy.misc, numpy, shutil, os, nibabel
import sys, getopt
import zipfile, os

In [10]:
#unzip nii package
package_path = input('Path to nii zip: ')
with zipfile.ZipFile(package_path, "r") as zip_ref:
    zip_ref.extractall("./Data/Nii")

Path to nii zip: ./Data/matthewl225-20190721_221951.zip


In [15]:
import gzip

#unzip individual run nii pakcages
nii_package_path = "./Data/Nii"
output_path = "./Data/Nii2"

if not os.path.exists(output_path):
    os.makedirs(output_path)
    print("Created ouput directory: " + output_path)

for root, directories, filenames in os.walk(nii_package_path):
    for filename in filenames:
        run_package = os.path.join(root, filename)
        with gzip.open(run_package, 'rb') as s_file, open(os.path.join(output_path, filename[:-3]), 'wb') as d_file:
            shutil.copyfileobj(s_file, d_file, 65536)
            #shutil.move(d_file, output_path)
                

In [16]:
shutil.rmtree("./Data/Nii")

FileNotFoundError: [WinError 3] The system cannot find the path specified: './Data/Nii'

In [18]:
#convert nii to png (borrowed from nii2png.py)
outputfile = "./Data/Images"
rotate_angle = int(input('Enter rotate angle (0 90 180 270): '))

if not (rotate_angle == 0 or rotate_angle == 90 or rotate_angle == 180 or rotate_angle == 270):
    print('You must enter a value that is either 90, 180, or 270. Quitting...')
    sys.exit()

data_path = "./Data/Nii2"
for root, directories, filenames in os.walk(data_path):
    for filename in filenames:
        nii_path = os.path.join(root, filename)
        image_array = nibabel.load(nii_path).get_data()
        print("Input file is ", nii_path)
        #print(len(image_array.shape))
        
        
        # else if 3D image inputted
        if len(image_array.shape) == 3:
            # set 4d array dimension values
            nx, ny, nz = image_array.shape

            if not os.path.exists(outputfile):
                os.makedirs(outputfile)
                print("Created ouput directory: " + outputfile)
            
            image_folder = os.path.join(outputfile, filename[:-8])
            
            if not os.path.exists(image_folder):
                os.makedirs(image_folder)
                print("Created ouput directory: " + image_folder)
            
            print('Reading NIfTI file...')

            total_slices = image_array.shape[2]

            slice_counter = 0
            # iterate through slices
            for current_slice in range(0, total_slices):
                # alternate slices
                if (slice_counter % 1) == 0:
                    # rotate or no rotate
                    if rotate_angle == 90 or rotate_angle == 180 or rotate_angle == 270:
                        if rotate_angle == 90:
                            data = numpy.rot90(image_array[:, :, current_slice])
                        elif ask_rotate_num == 180:
                            data = numpy.rot90(numpy.rot90(image_array[:, :, current_slice]))
                        elif ask_rotate_num == 270:
                            data = numpy.rot90(numpy.rot90(numpy.rot90(image_array[:, :, current_slice])))
                    else:
                        data = image_array[:, :, current_slice]

                    #alternate slices and save as png
                    if (slice_counter % 1) == 0:
                        #print('Saving image...')
                        image_name = nii_path[:-4] + "_z" + "{:0>3}".format(str(current_slice+1))+ ".png"
                        scipy.misc.imsave(image_name, data)
                        #print('Saved.')

                        #move images to folder
                        #print('Moving image...')
                        src = image_name
                        shutil.move(src, image_folder)
                        slice_counter += 1
                        #print('Moved.')

            print('Finished converting {}'.format(filename))
        else:
            print('Not a 3D Image. Please try again.')
        




Enter rotate angle (0 90 180 270): 90
Input file is  ./Data/Nii2\sub-OAS30001_ses-d0129_run-01_T1w.nii
Created ouput directory: ./Data/Images
Created ouput directory: ./Data/Images\sub-OAS30001_ses-d0129_run-01
Reading NIfTI file...


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:57: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.


Finished converting sub-OAS30001_ses-d0129_run-01_T1w.nii
Input file is  ./Data/Nii2\sub-OAS30001_ses-d0129_run-02_T1w.nii
Created ouput directory: ./Data/Images\sub-OAS30001_ses-d0129_run-02
Reading NIfTI file...
Finished converting sub-OAS30001_ses-d0129_run-02_T1w.nii
Input file is  ./Data/Nii2\sub-OAS30001_ses-d0757_run-01_T1w.nii
Created ouput directory: ./Data/Images\sub-OAS30001_ses-d0757_run-01
Reading NIfTI file...
Finished converting sub-OAS30001_ses-d0757_run-01_T1w.nii
Input file is  ./Data/Nii2\sub-OAS30001_ses-d0757_run-02_T1w.nii
Created ouput directory: ./Data/Images\sub-OAS30001_ses-d0757_run-02
Reading NIfTI file...
Finished converting sub-OAS30001_ses-d0757_run-02_T1w.nii
Input file is  ./Data/Nii2\sub-OAS30001_ses-d2430_T1w.nii
Created ouput directory: ./Data/Images\sub-OAS30001_ses-d2430
Reading NIfTI file...
Finished converting sub-OAS30001_ses-d2430_T1w.nii
Input file is  ./Data/Nii2\sub-OAS30001_ses-d3132_T1w.nii
Created ouput directory: ./Data/Images\sub-OAS300

Reading NIfTI file...
Finished converting sub-OAS30006_ses-d3386_T1w.nii
Input file is  ./Data/Nii2\sub-OAS30007_ses-d0061_run-01_T1w.nii
Created ouput directory: ./Data/Images\sub-OAS30007_ses-d0061_run-01
Reading NIfTI file...
Finished converting sub-OAS30007_ses-d0061_run-01_T1w.nii
Input file is  ./Data/Nii2\sub-OAS30007_ses-d0061_run-02_T1w.nii
Created ouput directory: ./Data/Images\sub-OAS30007_ses-d0061_run-02
Reading NIfTI file...
Finished converting sub-OAS30007_ses-d0061_run-02_T1w.nii
Input file is  ./Data/Nii2\sub-OAS30007_ses-d1636_T1w.nii
Created ouput directory: ./Data/Images\sub-OAS30007_ses-d1636
Reading NIfTI file...
Finished converting sub-OAS30007_ses-d1636_T1w.nii
Input file is  ./Data/Nii2\sub-OAS30007_ses-d1641_T1w.nii
Created ouput directory: ./Data/Images\sub-OAS30007_ses-d1641
Reading NIfTI file...
Finished converting sub-OAS30007_ses-d1641_T1w.nii
Input file is  ./Data/Nii2\sub-OAS30007_ses-d1981_T1w.nii
Created ouput directory: ./Data/Images\sub-OAS30007_ses

Created ouput directory: ./Data/Images\sub-OAS30018_ses-d0070_run-01
Reading NIfTI file...
Finished converting sub-OAS30018_ses-d0070_run-01_T1w.nii
Input file is  ./Data/Nii2\sub-OAS30018_ses-d0070_run-02_T1w.nii
Created ouput directory: ./Data/Images\sub-OAS30018_ses-d0070_run-02
Reading NIfTI file...
Finished converting sub-OAS30018_ses-d0070_run-02_T1w.nii
Input file is  ./Data/Nii2\sub-OAS30018_ses-d0893_T1w.nii
Created ouput directory: ./Data/Images\sub-OAS30018_ses-d0893
Reading NIfTI file...
Finished converting sub-OAS30018_ses-d0893_T1w.nii
Input file is  ./Data/Nii2\sub-OAS30019_ses-d0376_T1w.nii
Created ouput directory: ./Data/Images\sub-OAS30019_ses-d0376
Reading NIfTI file...
Finished converting sub-OAS30019_ses-d0376_T1w.nii
Input file is  ./Data/Nii2\sub-OAS30020_ses-d0092_run-01_T1w.nii
Created ouput directory: ./Data/Images\sub-OAS30020_ses-d0092_run-01
Reading NIfTI file...
Finished converting sub-OAS30020_ses-d0092_run-01_T1w.nii
Input file is  ./Data/Nii2\sub-OAS300

...

Created ouput directory: ./Data/Images\sub-OAS31103_ses-d1829_run-02
Reading NIfTI file...
Finished converting sub-OAS31103_ses-d1829_run-02_T1w.nii
Input file is  ./Data/Nii2\sub-OAS31103_ses-d3821_T1w.nii
Created ouput directory: ./Data/Images\sub-OAS31103_ses-d3821
Reading NIfTI file...
Finished converting sub-OAS31103_ses-d3821_T1w.nii
Input file is  ./Data/Nii2\sub-OAS31104_ses-d1321_T1w.nii
Created ouput directory: ./Data/Images\sub-OAS31104_ses-d1321
Reading NIfTI file...
Finished converting sub-OAS31104_ses-d1321_T1w.nii
Input file is  ./Data/Nii2\sub-OAS31104_ses-d2333_T1w.nii
Created ouput directory: ./Data/Images\sub-OAS31104_ses-d2333
Reading NIfTI file...
Finished converting sub-OAS31104_ses-d2333_T1w.nii
Input file is  ./Data/Nii2\sub-OAS31105_ses-d0259_run-01_T1w.nii
Created ouput directory: ./Data/Images\sub-OAS31105_ses-d0259_run-01
Reading NIfTI file...
Finished converting sub-OAS31105_ses-d0259_run-01_T1w.nii
Input file is  ./Data/Nii2\sub-OAS31105_ses-d0259_run-02_T

Created ouput directory: ./Data/Images\sub-OAS31114_ses-d3002
Reading NIfTI file...
Finished converting sub-OAS31114_ses-d3002_T1w.nii
Input file is  ./Data/Nii2\sub-OAS31114_ses-d3338_T1w.nii
Created ouput directory: ./Data/Images\sub-OAS31114_ses-d3338
Reading NIfTI file...
Finished converting sub-OAS31114_ses-d3338_T1w.nii
Input file is  ./Data/Nii2\sub-OAS31114_ses-d3479_T1w.nii
Created ouput directory: ./Data/Images\sub-OAS31114_ses-d3479
Reading NIfTI file...
Finished converting sub-OAS31114_ses-d3479_T1w.nii
Input file is  ./Data/Nii2\sub-OAS31115_ses-d0466_T1w.nii
Created ouput directory: ./Data/Images\sub-OAS31115_ses-d0466
Reading NIfTI file...
Finished converting sub-OAS31115_ses-d0466_T1w.nii
Input file is  ./Data/Nii2\sub-OAS31116_ses-d0329_run-01_T1w.nii
Created ouput directory: ./Data/Images\sub-OAS31116_ses-d0329_run-01
Reading NIfTI file...
Finished converting sub-OAS31116_ses-d0329_run-01_T1w.nii
Input file is  ./Data/Nii2\sub-OAS31116_ses-d0329_run-02_T1w.nii
Created

Created ouput directory: ./Data/Images\sub-OAS31132_ses-d2580_run-02
Reading NIfTI file...
Finished converting sub-OAS31132_ses-d2580_run-02_T1w.nii
Input file is  ./Data/Nii2\sub-OAS31132_ses-d2580_run-03_T1w.nii
Created ouput directory: ./Data/Images\sub-OAS31132_ses-d2580_run-03
Reading NIfTI file...
Finished converting sub-OAS31132_ses-d2580_run-03_T1w.nii
Input file is  ./Data/Nii2\sub-OAS31132_ses-d2580_run-04_T1w.nii
Created ouput directory: ./Data/Images\sub-OAS31132_ses-d2580_run-04
Reading NIfTI file...
Finished converting sub-OAS31132_ses-d2580_run-04_T1w.nii
Input file is  ./Data/Nii2\sub-OAS31132_ses-d3163_run-01_T1w.nii
Created ouput directory: ./Data/Images\sub-OAS31132_ses-d3163_run-01
Reading NIfTI file...
Finished converting sub-OAS31132_ses-d3163_run-01_T1w.nii
Input file is  ./Data/Nii2\sub-OAS31132_ses-d3163_run-02_T1w.nii
Created ouput directory: ./Data/Images\sub-OAS31132_ses-d3163_run-02
Reading NIfTI file...
Finished converting sub-OAS31132_ses-d3163_run-02_T1w

Created ouput directory: ./Data/Images\sub-OAS31144_ses-d0072_run-01
Reading NIfTI file...
Finished converting sub-OAS31144_ses-d0072_run-01_T1w.nii
Input file is  ./Data/Nii2\sub-OAS31144_ses-d0072_run-02_T1w.nii
Created ouput directory: ./Data/Images\sub-OAS31144_ses-d0072_run-02
Reading NIfTI file...
Finished converting sub-OAS31144_ses-d0072_run-02_T1w.nii
Input file is  ./Data/Nii2\sub-OAS31145_ses-d0483_run-01_T1w.nii
Created ouput directory: ./Data/Images\sub-OAS31145_ses-d0483_run-01
Reading NIfTI file...
Finished converting sub-OAS31145_ses-d0483_run-01_T1w.nii
Input file is  ./Data/Nii2\sub-OAS31145_ses-d0483_run-02_T1w.nii
Created ouput directory: ./Data/Images\sub-OAS31145_ses-d0483_run-02
Reading NIfTI file...
Finished converting sub-OAS31145_ses-d0483_run-02_T1w.nii
Input file is  ./Data/Nii2\sub-OAS31146_ses-d0073_T1w.nii
Created ouput directory: ./Data/Images\sub-OAS31146_ses-d0073
Reading NIfTI file...
Finished converting sub-OAS31146_ses-d0073_T1w.nii
Input file is  .

Finished converting sub-OAS31159_ses-d0074_run-01_T1w.nii
Input file is  ./Data/Nii2\sub-OAS31159_ses-d0074_run-02_T1w.nii
Created ouput directory: ./Data/Images\sub-OAS31159_ses-d0074_run-02
Reading NIfTI file...
Finished converting sub-OAS31159_ses-d0074_run-02_T1w.nii
Input file is  ./Data/Nii2\sub-OAS31159_ses-d1019_T1w.nii
Created ouput directory: ./Data/Images\sub-OAS31159_ses-d1019
Reading NIfTI file...
Finished converting sub-OAS31159_ses-d1019_T1w.nii
Input file is  ./Data/Nii2\sub-OAS31159_ses-d1893_T1w.nii
Created ouput directory: ./Data/Images\sub-OAS31159_ses-d1893
Reading NIfTI file...
Finished converting sub-OAS31159_ses-d1893_T1w.nii
Input file is  ./Data/Nii2\sub-OAS31160_ses-d6062_T1w.nii
Created ouput directory: ./Data/Images\sub-OAS31160_ses-d6062
Reading NIfTI file...
Finished converting sub-OAS31160_ses-d6062_T1w.nii
Input file is  ./Data/Nii2\sub-OAS31161_ses-d0056_T1w.nii
Created ouput directory: ./Data/Images\sub-OAS31161_ses-d0056
Reading NIfTI file...
Finishe

In [19]:
runs_list = next(os.walk('./Data/Images'))[1]
#print(runs_list)

In [54]:
import math

AD_list = []

for run in runs_list:
    # if dx1 starts with AD then patient has AD
    # either 0 or 1
    # print(run)
    subject_regex = re.compile("OAS(?P<order>[0-9]+)")
    subject = subject_regex.search(run).group(1)
    
    date_regex = re.compile("d(?P<order>[0-9]+)")
    date = date_regex.search(run).group(1)
    
    #print(subject, date)
    
    subject_df = df.loc[df["Subject"] == subject]
    #print(subject_df)
    
    #label = min(list(map(int, subject_df['Date'])), key=lambda x:(int(date)-x))
    df_dates = subject_df['Date']
    #print(type(df_dates))
    
    if int(date) > df_dates.iloc[-1]:
        label_date = df_dates.iloc[-1]
    else:
        label_date = min(i for i in df_dates if i >= int(date))
    
    label = subject_df.loc[df['Date'] == label_date, 'dx1'].item()
    if type(label) == float:
        print(label)
        runs_list.remove(run)
        continue
    AD = 1 if label.startswith('AD') else 0
    AD_list.append(AD)
    
    #print(run, AD)

Labelsdf = pd.DataFrame()
#df = pd.DataFrame(columns=['Run', 'AD'])
Labelsdf['Run'] = pd.Series(runs_list)
Labelsdf['AD'] = pd.Series(AD_list)

In [49]:
print(label)

Cognitively normal


In [56]:
Labelsdf

,Run,AD
0,sub-OAS30001_ses-d0129_run-01,0
1,sub-OAS30001_ses-d0129_run-02,0
2,sub-OAS30001_ses-d0757_run-01,0
3,sub-OAS30001_ses-d0757_run-02,0
4,sub-OAS30001_ses-d2430,0
5,sub-OAS30001_ses-d3132,0
6,sub-OAS30002_ses-d0371,0
7,sub-OAS30002_ses-d0653_run-01,0
8,sub-OAS30002_ses-d0653_run-02,0
9,sub-OAS30002_ses-d2340_run-01,0


In [57]:
Labelsdf.to_csv("./Data/Labels.csv")

In [58]:
Labelsdf.groupby("AD").size()

AD
0    2854
1     535
dtype: int64